In [1]:
from sklearn.metrics import mean_absolute_error as MSE
from tensorflow.keras.models import Sequential
from cmcrameri import cm
from tqdm import tqdm
import seaborn as sns
from common import *
from models import *

# Loading our data

In [2]:
# df = pd.read_csv("../data/input_data/MAIN_DATASET.csv")
# price = df['NO2_price'].values.reshape(-1,1)
# price_days = seperate_column_to_days(price)

# Structuring our data

In [3]:
# lookbehind = 7
# input_width = lookbehind*24
# horizon = 24
# no_hours = input_width + horizon
# stride = 24
# hour_in_days = int(no_hours / stride)

# price_dataset = []


# for i in range(len(price_days) - hour_in_days+1):
#     price_dataset.append(np.concatenate((price_days[i:i+hour_in_days])))


# price_dataset = np.array(price_dataset)

# scaler = MinMaxScaler()
# price_dataset = scaler.fit_transform(price_dataset[:,:,0])
# price_dataset = price_dataset[..., np.newaxis].astype(np.float32)



# n,m,k = price_dataset.shape

# print(price_dataset.shape)


In [4]:
# print(price_dataset.shape)
# testing = scaler.fit_transform(price_dataset)


# Train test split

In [5]:
# train = int(0.7*n)
# valid = int(0.9*n)
# X_train = price_dataset[:train, :input_width]
# X_valid = price_dataset[train:valid, :input_width]
# X_test = price_dataset[valid:, :input_width]

# Y = np.empty((n, input_width, horizon))
# for step_ahead in range(1, horizon + 1):
#     Y[:,:, step_ahead - 1] = price_dataset[:,step_ahead:step_ahead + input_width, 0]

# Y_train = Y[:train]
# Y_valid = Y[train:valid]
# Y_test = Y[valid:]

In [6]:
# print(X_train.shape)

In [7]:
# Training data
dataset_df = pd.read_csv("../data/input_data/MAIN_DATASET.csv")[["NO2_price", "NO2_load_actual","NO2_load_delta", "NO2_generation_actual", "NO2_generation_delta", "dato_id","date_time"]][:-7381]; print(dataset_df.shape)

display(dataset_df.tail(3))


scaler = MinMaxScaler()
print("***",dataset_df["NO2_price"].values.reshape(-1,1).shape)

dataset_df["NO2_price"] = scaler.fit_transform(dataset_df["NO2_price"].values.reshape(-1,1))
dataset_df["NO2_load_actual"] = scaler.fit_transform(dataset_df["NO2_load_actual"].values.reshape(-1,1))
dataset_df["NO2_load_delta"] = scaler.fit_transform(dataset_df["NO2_load_delta"].values.reshape(-1,1))
dataset_df["NO2_generation_actual"] = scaler.fit_transform(dataset_df["NO2_generation_actual"].values.reshape(-1,1))
dataset_df["NO2_generation_delta"] = scaler.fit_transform(dataset_df["NO2_generation_delta"].values.reshape(-1,1))


(44267, 7)


,NO2_price,NO2_load_actual,NO2_load_delta,NO2_generation_actual,NO2_generation_delta,dato_id,date_time
44264,35.69,5562,-256,15808,-7655,2021-01-20 20:00:00+01:00,2021-01-20-20
44265,32.19,5389,-12,13936,-6872,2021-01-20 21:00:00+01:00,2021-01-20-21
44266,29.49,5197,39,12724,-6238,2021-01-20 22:00:00+01:00,2021-01-20-22


*** (44267, 1)


In [8]:
def create_time_series_data(raw_train_df, nbf_features, nbf_outputs, column_key, column_suffix="", include_target_outputs=True):
    nbf_columns = nbf_features + nbf_outputs
    time_series_df, time_series = sliding_window_algorithm(raw_train_df, nbf_columns, nbf_features, nbf_outputs, column_key, column_suffix, include_target_outputs)
    return time_series, time_series_df


def sliding_window_algorithm(raw_data_df, nbf_columns, input_window, output_window, column_key, column_suffix, include_target_outputs, string_array=False):
    raw_data = raw_data_df[column_key].to_numpy()
    dim0 = raw_data.shape[0] - (nbf_columns - 1) # Number of time-slices
    dim1 = nbf_columns # Time-step length including input window and output window
    
    if string_array:
        data = np.empty((dim0,dim1), dtype=object) # Time series matrix
    else:
        data = np.zeros((dim0,dim1)) # Time series matrix

    data[0, :] = raw_data[:dim1] # First initial sample/time-slice (n equals 0)
    n = 1 # n is the count variable
    
    # Looping through the initial input data and constucting the 
    # time series data based on the sliding window approach. 
    # See the report for more descriptions
    for j in range(dim1, raw_data.shape[0]):
        data[n, :-1] = data[n-1, 1:]
        data[n, nbf_columns-1:] = raw_data[j]
        n += 1

    if include_target_outputs:
        # Creating dataframe    
        feature_names = [f"x{i}_{column_suffix}" for i in range(1,input_window+1)]
        if(output_window > 1):
            output_names = [f"y{i}_{column_suffix}" for i in range(1,output_window+1)]    
        else:
            output_names = [f"y_{column_suffix}"]
        
        time_series_df = pd.DataFrame(data=data, columns=feature_names+output_names)
    else:
        feature_names = [f"x{i}_{column_suffix}" for i in range(1,input_window+2)]
        time_series_df = pd.DataFrame(data=data, columns=feature_names)

    return time_series_df, data

In [9]:
# Time series slicing parameters
nbf_features = input_window = 1
nbf_outputs = output_window = horizon = 24
n_steps = time_steps = 72

# Creating time series traing and evaluation (test) data besed on sliding window approach
time_series, time_series_df = create_time_series_data(dataset_df,                                                                             
                                                      nbf_features, 
                                                      nbf_outputs, 
                                                      column_key="NO2_price", 
                                                      column_suffix="DA",
                                                      include_target_outputs=True)
#time_series_df.to_csv("TIMESERIES_VALIDATION.csv")

# Displaying time series data
time_series_df.index.name = "t"

print(time_series.shape)
print(time_series_df.shape)
# print(time_series_df.columns)

X_train, y_train = time_series[ :34272, :input_window], time_series[ :34272, input_window:]
X_valid, y_valid = time_series[34272:34272+6581, :input_window], time_series[34272:34272+6581, input_window:]
X_test, y_test = time_series[34272+6581:, :input_window], time_series[34272+6581:, input_window:]

# X_train = np.expand_dims(X_train, axis=1)
# X_valid = np.expand_dims(X_valid, axis=1)
# X_test = np.expand_dims(X_test, axis=1)

# print(X_train.shape, y_train.shape)
# print(X_valid.shape, y_valid.shape)
# print(X_test.shape, y_test.shape)

print("before")
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)
print(X_test.shape, y_test.shape)

train_start_idx = X_train.shape[0]%time_steps
X_train = X_train[train_start_idx: , :] # 
y_train = y_train[train_start_idx: , :] # 

batch_size_train = X_train.shape[0] // time_steps
x_train_RNN = X_train.reshape((batch_size_train, time_steps, nbf_features))
y_train_RNN = y_train.copy()
y_train_RNN = y_train_RNN.reshape((batch_size_train, time_steps, nbf_outputs))


# Validation 
valid_start_idx = X_valid.shape[0]%time_steps
X_valid = X_valid[valid_start_idx: , :] # 
y_valid = y_valid[valid_start_idx: , :] # 

batch_size_valid = X_valid.shape[0] // time_steps
x_valid_RNN = X_valid.reshape((batch_size_valid, time_steps, nbf_features))
y_valid_RNN = y_valid.copy()
y_valid_RNN = y_valid_RNN.reshape((batch_size_valid, time_steps, nbf_outputs))

# Test 
test_start_idx = X_test.shape[0]%time_steps
X_test = X_test[test_start_idx: , :] # 
y_test = y_test[test_start_idx: , :] # 

batch_size_test = X_test.shape[0] // time_steps
x_test_RNN = X_test.reshape((batch_size_test, time_steps, nbf_features))
y_test_RNN = y_test.copy()
y_test_RNN = y_test_RNN.reshape((batch_size_test, time_steps, nbf_outputs))

X_train = x_train_RNN#.swapaxes(1,2)
y_train = y_train_RNN#.squeeze()   

X_valid = x_valid_RNN#.swapaxes(1,2)   
y_valid = y_valid_RNN#.squeeze()   

X_test = x_test_RNN#.swapaxes(1,2)   
y_test = y_test_RNN#.squeeze()

print("After")
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)
print(X_test.shape, y_test.shape)

(44243, 25)
(44243, 25)
before
(34272, 1) (34272, 24)
(6581, 1) (6581, 24)
(3390, 1) (3390, 24)
After
(476, 72, 1) (476, 72, 24)
(91, 72, 1) (91, 72, 24)
(47, 72, 1) (47, 72, 24)


## Hyperparameters

In [10]:
# Common
epochs = 25
batch_size = [100, 200, 300]
eta_list = np.logspace(-2, -4, 3)
neurons_list = [128, 64, 32]
filters = 64
kernel_size = 3
strides = 3

learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                            patience=3, 
                                                            verbose=1, 
                                                            factor=0.5, 
                                                            min_lr=0.000001)
callbacks = [learning_rate_reduction]

# Setting up the univariate SimpleRNN

In [ ]:
k=horizon
class RNN(Sequential):
    def __init__(self, eta, neurons, horizon=24):
        super(self.__class__, self).__init__()

        # RNN architecture
        self.add(keras.layers.SimpleRNN(neurons, return_sequences=True, input_shape=[None, 1]))
        self.add(keras.layers.SimpleRNN(neurons, return_sequences=True))
        self.add(keras.layers.Dense(horizon))

        # Model compile settings
        opt = keras.optimizers.Adam(learning_rate=eta)

        # Compile model
        self.compile(loss='mse', optimizer=opt)


# return_sequences=True --> (256, 168, 24)
# return_sequences=False --> (256, 24)

## Grid search

In [ ]:
results_columns = ["model","batch_size", "learning_rate", "epoch", "neurons", "mse_all"]
results_columns += [f"mse{i}" for i in range(1,horizon+1)] 
results = pd.DataFrame(columns=results_columns)
input_width = time_steps
x_columns = [f"x{k}" for k in range(1, input_width+1)]
t_columns = [f"t{k}" for k in range(1, horizon+1)]
t_hat_columns = [f"t_hat{k}" for k in range(1, horizon+1)]
preds_columns = columns=x_columns+t_columns+t_hat_columns
results_preds = pd.DataFrame(columns=preds_columns)

epochs_colmns = [f"epoch{i}" for i in range(epochs)]
Y_test_last = y_test[:,-1]
best_mse = np.inf


i = 0
for eta in tqdm(eta_list):
    for batch in batch_size:
        for neurons in neurons_list:
            RNN_model = RNN(eta, neurons, horizon=horizon)

            # Training
            history = RNN_model.fit(X_train, y_train, 
                        epochs=epochs, batch_size = batch, 
                        shuffle=False, callbacks= callbacks,  
                        validation_data=(X_valid, y_valid), verbose=0)

            # Prediction
            Y_pred_rnn = RNN_model.predict(X_test)
            Y_pred_rnn_last = Y_pred_rnn[:,-1]     
            
            # Results
            mse_val = MSE(Y_test_last, Y_pred_rnn_last)
            mse_all = np.mean((Y_test_last - Y_pred_rnn_last) ** 2, axis=0) # The mean of each column 
            
            
            # Storing results for all models
            res1 = np.array([i, batch, eta, epochs, neurons, mse_val])
            res2 = np.array(mse_all)
            res = np.concatenate([res1,res2], axis=0)
            results.loc[i] = res
            results.to_csv(f"{REPORT_DATA}univariate_simpleRNN_results.csv")

            # Storing results from the best model
            if best_mse > mse_val:
                best_mse = mse_val
                print(f"\nNew best model with MSE = {mse_val} !\n")
                # Predictions and target for best results
                preds_res =  np.concatenate([X_test[...,0], Y_test_last, Y_pred_rnn_last],axis=1)
                best_results_preds = pd.DataFrame(preds_res, columns=preds_columns)
                best_results_preds.to_csv(f"{REPORT_DATA}best_models/univariate_simpleRNN_preds_best.csv")

                
                # Best results relative to parameters
                best_results = pd.DataFrame(columns=results_columns)
                best_results.loc[i] = res
                best_results.to_csv(f"{REPORT_DATA}best_models/univariate_simpleRNN_results_best.csv")
                
                # Model evaluation result            
                best_train_val_losses = pd.DataFrame(columns=["training_loss", "validation_loss"])
                best_train_val_losses["training_loss"] = history.history["loss"]
                best_train_val_losses["validation_loss"] = history.history["val_loss"]
                best_train_val_losses.to_csv(f"{REPORT_DATA}best_models/univariate_simpleRNN_train_val_losses_best.csv")            
                
            i += 1

In [11]:
k=1

# Setting up the univariate ConvGRU

In [ ]:
class ConvGRU(Sequential):
    def __init__(self, eta, neurons_first, neurons_second, horizon=24):
        super(self.__class__, self).__init__()

        # ConvGRU architecture
        self.add(keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, strides=strides, padding="valid", input_shape=[None, k]))
        self.add(keras.layers.GRU(neurons_first, return_sequences=True))
        self.add(keras.layers.GRU(neurons_second, return_sequences=True))
        self.add(keras.layers.Dense(horizon))

        # Model compile settings
        opt = keras.optimizers.Adam(learning_rate=eta)

        # Compile model
        self.compile(loss='mse', optimizer=opt)

## Grid search

In [ ]:
results_columns = ["model","batch_size", "learning_rate", "epoch", "neurons", "mse_all"]
results_columns += [f"mse{i}" for i in range(1,horizon+1)] 
results = pd.DataFrame(columns=results_columns)

x_columns = [f"x{k}" for k in range(1, input_width+1)]
t_columns = [f"t{k}" for k in range(1, horizon+1)]
t_hat_columns = [f"t_hat{k}" for k in range(1, horizon+1)]
preds_columns = columns=x_columns+t_columns+t_hat_columns
results_preds = pd.DataFrame(columns=preds_columns)

epochs_colmns = [f"epoch{i}" for i in range(epochs)]
Y_test_last = y_test[:,-1]
best_mse = np.inf

i = 0
for eta in tqdm(eta_list):
    for batch in batch_size:
        for neurons in neurons_list:
            ConvGRU_model = ConvGRU(eta, neurons, neurons, horizon=horizon)

            # Training
            history = ConvGRU_model.fit(X_train, y_train[:,2::strides], 
                        epochs=epochs, batch_size = batch, 
                        shuffle=False, callbacks= callbacks,  
                        validation_data=(X_valid, y_valid[:,2::strides]), verbose=0)

            # Prediction
            Y_pred_rnn = ConvGRU_model.predict(X_test)
            Y_pred_rnn_last = Y_pred_rnn[:,-1]     
            
            # Results
            mse_val = MSE(Y_test_last, Y_pred_rnn_last)
            mse_all = np.mean((Y_test_last - Y_pred_rnn_last) ** 2, axis=0) # The mean of each column 
                        
            # Storing results for all models
            res1 = np.array([i, batch, eta, epochs, neurons, mse_val])
            res2 = np.array(mse_all)
            res = np.concatenate([res1,res2], axis=0)
            results.loc[i] = res
            results.to_csv(f"{REPORT_DATA}univariate_convGRU_results.csv")

            # Storing results from the best model
            if best_mse > mse_val:
                best_mse = mse_val
                print(f"\nNew best model with MSE = {mse_val} !\n")
                # Predictions and target for best results
                preds_res =  np.concatenate([X_test[...,0], Y_test_last, Y_pred_rnn_last],axis=1)
                best_results_preds = pd.DataFrame(preds_res, columns=preds_columns)
                best_results_preds.to_csv(f"{REPORT_DATA}best_models/univariate_convGRU_preds_best.csv")

                
                # Best results relative to parameters
                best_results = pd.DataFrame(columns=results_columns)
                best_results.loc[i] = res
                best_results.to_csv(f"{REPORT_DATA}best_models/univariate_convGRU_results_best.csv")
                
                # Model evaluation result            
                best_train_val_losses = pd.DataFrame(columns=["training_loss", "validation_loss"])
                best_train_val_losses["training_loss"] = history.history["loss"]
                best_train_val_losses["validation_loss"] = history.history["val_loss"]
                best_train_val_losses.to_csv(f"{REPORT_DATA}best_models/univariate_convGRU_train_val_losses_best.csv")            
                
            i += 1

# Setting up the univariate LSTM

In [12]:
class LSTM(Sequential):
    def __init__(self, eta, neurons, horizon=24):
        super(self.__class__, self).__init__()
        
        # self.add(keras.layers.Bidirectional(keras.layers.LSTM(neurons, return_sequences=True)))
        # self.add(keras.layers.Bidirectional(keras.layers.LSTM(neurons, return_sequences=True)))
        
        self.add(keras.layers.LSTM(neurons, return_sequences=True, input_shape=[None, k]))
        self.add(keras.layers.LSTM(neurons, return_sequences=True))
        self.add(keras.layers.Dense(horizon))

        # Model compile settings
        opt = keras.optimizers.Adam(learning_rate=eta)

        # Compile model
        self.compile(loss='mse', optimizer=opt)

## Grid search

In [15]:
results_columns = ["model","batch_size", "learning_rate", "epoch", "neurons", "mse_all"]
results_columns += [f"mse{i}" for i in range(1,horizon+1)] 
results = pd.DataFrame(columns=results_columns)

input_width = time_steps
x_columns = [f"x{k}" for k in range(1, input_width+1)]
t_columns = [f"t{k}" for k in range(1, horizon+1)]
t_hat_columns = [f"t_hat{k}" for k in range(1, horizon+1)]
preds_columns = columns=x_columns+t_columns+t_hat_columns
results_preds = pd.DataFrame(columns=preds_columns)

epochs_colmns = [f"epoch{i}" for i in range(epochs)]

Y_test_last = y_test[:,-1]

best_mse = np.inf

i = 0
for eta in tqdm(eta_list):
    for batch in batch_size:
        for neurons in neurons_list:
            LSTM_model = LSTM(eta, neurons, horizon=horizon)

            # Training
            history = LSTM_model.fit(X_train, y_train, 
                        epochs=epochs, batch_size = batch, 
                        shuffle=False, callbacks= callbacks,  
                        validation_data=(X_valid, y_valid), verbose=0)

            # Prediction
            Y_pred_rnn = LSTM_model.predict(X_test)
            Y_pred_rnn_last = Y_pred_rnn[:,-1]     
            
            # Results
            mse_val = MSE(Y_test_last, Y_pred_rnn_last)
            mse_all = np.mean((Y_test_last - Y_pred_rnn_last) ** 2, axis=0) # The mean of each column 
            
            
            # Storing results for all models
            res1 = np.array([i, batch, eta, epochs, neurons, mse_val])
            res2 = np.array(mse_all)
            res = np.concatenate([res1,res2], axis=0)
            results.loc[i] = res
            results.to_csv(f"{REPORT_DATA}univariate_LSTM_results.csv")

            # Storing results from the best model
            if best_mse > mse_val:
                best_mse = mse_val
                print(f"\nNew best model with MSE = {mse_val} !\n")
                # Predictions and target for best results
                preds_res =  np.concatenate([X_test[...,0], Y_test_last, Y_pred_rnn_last],axis=1)
                best_results_preds = pd.DataFrame(preds_res, columns=preds_columns)
                best_results_preds.to_csv(f"{REPORT_DATA}best_models/univariate_LSTM_preds_best.csv")

                
                # Best results relative to parameters
                best_results = pd.DataFrame(columns=results_columns)
                best_results.loc[i] = res
                best_results.to_csv(f"{REPORT_DATA}best_models/univariate_LSTM_results_best.csv")
                
                # Model evaluation result            
                best_train_val_losses = pd.DataFrame(columns=["training_loss", "validation_loss"])
                best_train_val_losses["training_loss"] = history.history["loss"]
                best_train_val_losses["validation_loss"] = history.history["val_loss"]
                best_train_val_losses.to_csv(f"{REPORT_DATA}best_models/univariate_LSTM_train_val_losses_best.csv")            
                
            i += 1

  0%|          | 0/3 [00:00<?, ?it/s]


Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.005.

Epoch 00015: ReduceLROnPlateau reducing learning rate to 0.0025.

Epoch 00018: ReduceLROnPlateau reducing learning rate to 0.00125.

New best model with MSE = 0.04007991661110651 !


Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.005.

Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.0025.

Epoch 00010: ReduceLROnPlateau reducing learning rate to 0.00125.

Epoch 00013: ReduceLROnPlateau reducing learning rate to 0.000625.

Epoch 00024: ReduceLROnPlateau reducing learning rate to 0.0003125.

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.005.

Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.0025.

Epoch 00014: ReduceLROnPlateau reducing learning rate to 0.00125.

Epoch 00020: ReduceLROnPlateau reducing learning rate to 0.000625.

Epoch 00023: ReduceLROnPlateau reducing learning rate to 0.0003125.

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.005.

Epoch 00008: Red

 33%|███▎      | 1/3 [00:59<01:58, 59.18s/it]


Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005.

Epoch 00008: ReduceLROnPlateau reducing learning rate to 0.00025.

Epoch 00011: ReduceLROnPlateau reducing learning rate to 0.000125.

Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.0005.

Epoch 00010: ReduceLROnPlateau reducing learning rate to 0.00025.

Epoch 00013: ReduceLROnPlateau reducing learning rate to 0.000125.

Epoch 00008: ReduceLROnPlateau reducing learning rate to 0.0005.

Epoch 00011: ReduceLROnPlateau reducing learning rate to 0.00025.

Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.0005.

Epoch 00010: ReduceLROnPlateau reducing learning rate to 0.00025.

Epoch 00013: ReduceLROnPlateau reducing learning rate to 0.000125.

Epoch 00016: ReduceLROnPlateau reducing learning rate to 6.25e-05.

Epoch 00019: ReduceLROnPlateau reducing learning rate to 3.125e-05.

Epoch 00022: ReduceLROnPlateau reducing learning rate to 1.5625e-05.

Epoch 00025: ReduceLROnPlateau reducing learning rate to

 67%|██████▋   | 2/3 [01:56<00:58, 58.31s/it]


Epoch 00016: ReduceLROnPlateau reducing learning rate to 5e-05.

Epoch 00019: ReduceLROnPlateau reducing learning rate to 2.5e-05.

Epoch 00022: ReduceLROnPlateau reducing learning rate to 1.25e-05.

Epoch 00025: ReduceLROnPlateau reducing learning rate to 6.25e-06.

Epoch 00017: ReduceLROnPlateau reducing learning rate to 5e-05.

Epoch 00025: ReduceLROnPlateau reducing learning rate to 2.5e-05.

Epoch 00017: ReduceLROnPlateau reducing learning rate to 5e-05.

Epoch 00025: ReduceLROnPlateau reducing learning rate to 2.5e-05.

Epoch 00025: ReduceLROnPlateau reducing learning rate to 5e-05.

Epoch 00024: ReduceLROnPlateau reducing learning rate to 5e-05.


100%|██████████| 3/3 [02:54<00:00, 58.04s/it]
